# This jupyter notebook provides all the details of Alpaca API

[1. Get historical/live Data](#1)

[2. Monitor results - Alpaca account level](#2)

[3. Monitor results - Strategy level](#3)

[4. Submit order](#4)

[5. Run multiprocess](#5)

In [2]:
import alpaca_backtrader_api
import backtrader as bt
from datetime import datetime

ModuleNotFoundError: No module named 'alpaca_backtrader_api'

In [4]:
os.environ['APCA_API_KEY_ID'] = 'PKCXJIA0J6B064W65PG8'
os.environ['APCA_API_SECRET_KEY'] = 'e2qNpzFFqFhgOrEeSXipkCh4zi2UyVcFo2R1KKcD'
os.environ['APCA_API_BASE_URL'] = 'https://paper-api.alpaca.markets'
api = REST()

<a id='1'></a>
# Get Data

* Exhaustive list of symbols
* Downloading the data

In [ ]:
"""
You have 3 options:
 - backtest (IS_BACKTEST=True, IS_LIVE=False)
 - paper trade (IS_BACKTEST=False, IS_LIVE=False)
 - live trade (IS_BACKTEST=False, IS_LIVE=True)
"""
IS_BACKTEST = False
IS_LIVE = False
symbol = "AAPL"


class SmaCross1(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30,   # period for the slow moving average
        rsi_per=14,
        rsi_upper=65.0,
        rsi_lower=35.0,
        rsi_out=50.0,
        warmup=35
    )

    def log(self, txt, dt=None):
        dt = dt or self.data.datetime[0]
        dt = bt.num2date(dt)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_trade(self, trade):
        self.log("placing trade for {}. target size: {}".format(
            trade.getdataname(),
            trade.size))

    def notify_order(self, order):
        print(f"Order notification. status{order.getstatusname()}.")
        print(f"Order info. status{order.info}.")

    def notify_store(self, msg, *args, **kwargs):
        super().notify_store(msg, *args, **kwargs)
        self.log(msg)

    def stop(self):
        print('==================================================')
        print('Starting Value - %.2f' % self.broker.startingcash)
        print('Ending   Value - %.2f' % self.broker.getvalue())
        print('==================================================')

    def __init__(self):
        self.live_bars = False
        sma1 = bt.ind.SMA(self.data0, period=self.p.pfast)
        sma2 = bt.ind.SMA(self.data0, period=self.p.pslow)
        self.crossover = bt.ind.CrossOver(sma1, sma2)

        rsi = bt.indicators.RSI(period=self.p.rsi_per,
                                upperband=self.p.rsi_upper,
                                lowerband=self.p.rsi_lower)

        self.crossdown = bt.ind.CrossDown(rsi, self.p.rsi_upper)
        self.crossup = bt.ind.CrossUp(rsi, self.p.rsi_lower)

    def notify_data(self, data, status, *args, **kwargs):
        super().notify_data(data, status, *args, **kwargs)
        print('*' * 5, 'DATA NOTIF:', data._getstatusname(status), *args)
        if data._getstatusname(status) == "LIVE":
            self.live_bars = True

    def next(self):
        if not self.live_bars and not IS_BACKTEST:
            # only run code if we have live bars (today's bars).
            # ignore if we are backtesting
            return
        # if fast crosses slow to the upside
        if not self.positionsbyname[symbol].size:
            if self.crossover > 0 or self.crossup > 0:
                self.buy(data=data0, size=5)  # enter long

        # in the market & cross to the downside
        if self.positionsbyname[symbol].size:
            if self.crossover <= 0 or self.crossdown < 0:
                self.close(data=data0)  # close long position

In [5]:
if __name__ == '__main__':
    import logging
    logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)
    cerebro = bt.Cerebro()
    cerebro.addstrategy(SmaCross1)

    store = alpaca_backtrader_api.AlpacaStore(
        key_id=ALPACA_API_KEY,
        secret_key=ALPACA_SECRET_KEY,
        paper=not IS_LIVE,
    )

    DataFactory = store.getdata  # or use alpaca_backtrader_api.AlpacaData
    if IS_BACKTEST:
        data0 = DataFactory(dataname=symbol, historical=True,
                            fromdate=datetime(2015, 1, 1),
                            timeframe=bt.TimeFrame.Days,
                            data_feed='iex')

    else:
        data0 = DataFactory(dataname=symbol,
                            historical=False,
                            timeframe=bt.TimeFrame.Ticks,
                            backfill_start=False,
                            data_feed='iex')
        # or just alpaca_backtrader_api.AlpacaBroker()
        broker = store.getbroker()
        cerebro.setbroker(broker)
    cerebro.adddata(data0)

    if IS_BACKTEST:
        # backtrader broker set initial simulated cash
        cerebro.broker.setcash(100000.0)

    print('Starting Portfolio Value: {}'.format(cerebro.broker.getvalue()))
    cerebro.run()
    print('Final Portfolio Value: {}'.format(cerebro.broker.getvalue()))
    cerebro.plot()

In [6]:
result_df = pd.DataFrame()

print(result[0]) # cum_qty, price, symbol, transaction_time

Asset({   'class': 'us_equity',
    'easy_to_borrow': False,
    'exchange': 'OTC',
    'fractionable': True,
    'id': 'f669b6d7-73cf-4f46-a41e-9e17c9924abe',
    'maintenance_margin_requirement': 100,
    'marginable': False,
    'name': 'Aston Martin Lagonda Global Holdings plc American Depositary '
            'Receipts - Unsponsored',
    'shortable': False,
    'status': 'active',
    'symbol': 'ARGGY',
    'tradable': False})


In [7]:
result[1]

Asset({   'class': 'us_equity',
    'easy_to_borrow': False,
    'exchange': 'OTC',
    'fractionable': False,
    'id': '4f8635d9-59c2-4f13-b5f5-a86243fa0ea8',
    'maintenance_margin_requirement': 100,
    'marginable': False,
    'name': 'ABRAXAS PETE CORP COM NEW',
    'shortable': False,
    'status': 'active',
    'symbol': 'AXAS',
    'tradable': False})

In [8]:
type(result[0])

alpaca_trade_api.entity.Asset

In [9]:
result_df = pd.DataFrame(columns = ['class','exchange','symbol'])

In [10]:
class_list = []
exchange_list = []
symbol_list = []

for res in result:
    #print(res.class)
    #class_list.append(res.class)
    exchange_list.append(res.exchange)
    symbol_list.append(res.symbol)

result_df = pd.DataFrame({ 'exchange': exchange_list, 'Symbol': symbol_list})
print(result_df)

      exchange     Symbol
0          OTC      ARGGY
1          OTC       AXAS
2          OTC       BDRL
3          OTC      BRCNF
4          OTC      BTLCY
...        ...        ...
11079     FTXU   LINK/BTC
11080     FTXU    LTC/USD
11081     FTXU   LTC/USDT
11082     FTXU  PAXG/USDT
11083     FTXU  MATIC/USD

[11084 rows x 2 columns]


In [11]:
result_df.Symbol.unique()

array(['ARGGY', 'AXAS', 'BDRL', ..., 'LTC/USDT', 'PAXG/USDT', 'MATIC/USD'],
      dtype=object)

In [12]:
result_df.exchange.unique()

array(['OTC', 'NASDAQ', 'AMEX', 'NYSE', 'ARCA', 'BATS', 'FTXU'],
      dtype=object)

In [13]:
#Finding a symbol
result_df[result_df['Symbol'] == 'ES00']

,exchange,Symbol


In [14]:
#Data download
from alpaca_trade_api.rest import TimeFrame
historical_data = api.get_bars(
    symbol='TSLA', #any symbol is acceptable if it can be found in Alpaca API
    timeframe=TimeFrame.Hour, 
    start="2018-01-01T00:00:00-00:00",
    end="2018-02-01T00:00:00-00:00"
).df

In [15]:
historical_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2018-01-02 09:00:00+00:00,312.0000,312.00,311.80,311.80,500,4,311.871000
2018-01-02 10:00:00+00:00,311.5000,311.50,311.50,311.50,847,3,311.500000
2018-01-02 11:00:00+00:00,310.9900,311.89,310.99,311.89,1012,17,311.328360
2018-01-02 12:00:00+00:00,311.0000,311.29,310.75,311.00,11519,100,310.994353
2018-01-02 13:00:00+00:00,311.5000,312.00,310.99,311.50,8021,85,311.429299
...,...,...,...,...,...,...,...
2018-01-31 19:00:00+00:00,354.8695,355.27,351.57,353.23,717878,6836,353.168927
2018-01-31 20:00:00+00:00,353.1200,354.97,351.00,354.46,839081,10979,353.272195
2018-01-31 21:00:00+00:00,354.3100,354.61,353.75,354.00,764607,276,354.309591


<a id='2'></a>
# Monitoring Results - Account Level


## Items on Dashboard

* get_all_transactions : List of all transactions and full activity 
* realized_profit_df : Closed positions
* get_open_positions : Open positions  
* portfolio_changes_output : charts of PnL and Equity
* get_metrics : get all metrics

https://github.com/chunyip135/AlgoQuant_Dashboard
https://docs.google.com/spreadsheets/d/1hPxsuwdDvpZpQYG2yi_djwcDG1RDAw1RcS-QZ5prUPo/edit#gid=0
https://datastudio.google.com/u/0/reporting/632a4e76-496c-4ab5-8e8f-85296118e961/page/dZktC?s=olg3z5e2JEI

Error for crypto : https://forum.alpaca.markets/t/quantity-error-when-replacing-partially-filled-sell-order/4329/2


In [16]:
#get_all_transactions(api)
#api.get_portfolio_history(trade)
#api.list_positions()


In [19]:
# # result = api.get_activities()
# result = api.get_activities()

# result_df = pd.DataFrame()

# print(result)

In [20]:
#positions = get_open_positions(api)
#positions

## Function 1 - All orders and PnL

In [21]:
def get_all_transactions(api):
    result = api.get_activities()

    result_df = pd.DataFrame()

    #print(result[0]) # cum_qty, price, symbol, transaction_time

    result_df = pd.DataFrame(columns = ['Qty','price','symbol','transaction_time'])

    qty_list = []
    price_list = []
    symbol_list = []
    transaction_time_list = []
    type_list = []

    for res in result:
        qty_list.append(res.cum_qty)
        price_list.append(res.price)
        symbol_list.append(res.symbol)
        transaction_time_list.append(res.transaction_time)
        type_list.append(res.side)



    result_df = pd.DataFrame({
        'Qty': qty_list, 
        'Price': price_list, 
        'Symbol': symbol_list, 
        'Transaction_time': transaction_time_list, 
        'Type': type_list})

    return result_df

In [22]:
result_df = get_all_transactions(api)
result_df

,Qty,Price,Symbol,Transaction_time,Type
0,1,16195.41,BTC/USD,2022-11-21 06:01:19.581237+00:00,buy


In [23]:
100000 - (1133.37-1130.54)

99997.17

In [24]:
(1*16084.2885)-1*16126.29  + 100000

99957.9985

In [25]:
40.49


40.49

In [26]:
16195.41*(1-.9975)

40.488524999999136

In [27]:
api.list_positions() 

[Position({   'asset_class': 'crypto',
     'asset_id': '64bbff51-59d6-4b3c-9351-13ad85e3c752',
     'asset_marginable': False,
     'avg_entry_price': '16195.41',
     'change_today': '-0.0069204338987929',
     'cost_basis': '16154.921475',
     'current_price': '16561.29',
     'exchange': 'FTXU',
     'lastday_price': '16676.7',
     'market_value': '16519.886775',
     'qty': '0.9975',
     'qty_available': '0.9975',
     'side': 'long',
     'symbol': 'BTCUSD',
     'unrealized_intraday_pl': '-115.121475',
     'unrealized_intraday_plpc': '-0.0069204338987929',
     'unrealized_pl': '364.9653',
     'unrealized_plpc': '0.02259158613459'})]

In [28]:
#api.get_portfolio_history()

In [29]:
get_all_transactions(api)

,Qty,Price,Symbol,Transaction_time,Type
0,1,16195.41,BTC/USD,2022-11-21 06:01:19.581237+00:00,buy


In [30]:
def realized_profit_df(api):

    result_df = get_all_transactions(api)

    df_buy = result_df[result_df.Type == 'buy']

    df_sell = result_df[result_df.Type == 'sell']

    df = pd.merge(df_buy, df_sell, on = 'Symbol', how = 'right', suffixes = ['_buy','_sell'])

    #print(df_sell)

    testing = df_sell.sort_values(by = 'Transaction_time')

    testing['Price'] = testing.Price.astype('float')
    df_buy['Price'] = df_buy.Price.astype('float')

    output_frame = pd.DataFrame(columns = ['Symbol', 'Qty','Avg_cost','Avg_holding_period',
                                 'Earliest_buy_time','Latest_buy_time','Sell_time','Profit_per_unit','Total Profit', 'Winning_bet?'])

    for sym in testing.Symbol.unique():
        #print(sym)
        buy = df_buy.loc[df_buy.Symbol == sym]
        sell = testing.loc[testing.Symbol == sym]

        obs = [] # completed sell\'s index
        for i, row in sell.iterrows():
            output_dic = {}
            if i not in obs:
                out = buy.loc[(buy.Transaction_time < row.Transaction_time)]
                idx = [j for j in out.index if j not in obs]
                out = out.loc[idx]
                print(row, out.shape[0], int(round(float(row.Qty))))
                assert out.shape[0] == int(round(float(row.Qty)))
                
                #assert out.shape[0] == row.Qty
                # Avg_cost
                #print(row.Price - out.groupby('Symbol').Price.mean())
                #print(row.Transaction_time - out.groupby('Symbol').Transaction_time.mean())
                output_dict = {'Symbol': sym, 
                               'Qty': int(round(float(row.Qty))), 
                               'Avg_cost': round(out.groupby('Symbol').Price.mean()[0],2),
                               'Avg_holding_period': (row.Transaction_time - out.groupby('Symbol').Transaction_time.mean())[0],
                               'Earliest_buy_time': out.groupby('Symbol').Transaction_time.min()[0],
                               'Latest_buy_time': out.groupby('Symbol').Transaction_time.max()[0],
                               'Sell_time': row.Transaction_time,
                               'Profit_per_unit': round(row.Price - out.groupby('Symbol').Price.mean()[0],2),
                               'Total Profit': round((row.Price - out.groupby('Symbol').Price.mean())[0] * int(round(float(row.Qty))),2),
                               'Winning_bet?': True if round(row.Price - out.groupby('Symbol').Price.mean()[0],2) > 0 else False}
                output_frame = output_frame.append(output_dict, ignore_index = True)
                
                if len(idx) > 1:
                    for ix in idx:
                        obs.append(ix)
                else:
                    obs.append(idx[0])
            

    output_frame = output_frame.sort_values('Sell_time', ascending = False)
    output_frame['Avg_holding_period'] = output_frame['Avg_holding_period'].apply(lambda x: x.days)

    return output_frame

In [31]:
realized_profit_df(api)

,Symbol,Qty,Avg_cost,Avg_holding_period,Earliest_buy_time,Latest_buy_time,Sell_time,Profit_per_unit,Total Profit,Winning_bet?


### Function 2 - portfolio changes and metrics

In [32]:
from alpaca_trade_api.rest import REST, TimeFrame
import pandas as pd 
import numpy as np
import os
from alpaca_trade_api import REST

import datetime as dt

def get_trading_cal(api):
    cal = api.get_calendar()
    cal_ls = []
    for c in cal:
        cal_ls.append(c.date)
      
    cal_ls = pd.Series(cal_ls)
    return cal_ls

def get_portfolio_changes(api):
    
    acc_start = api.get_account().created_at.strftime('%Y-%m-%d') # the date where account was created

    cal_ls = get_trading_cal(api) # obtain trading calendar

    # Obtain the trading days that were between the account created date and today, then convert to date format
    trading_days = cal_ls[(cal_ls >= acc_start) & (cal_ls < dt.datetime.strftime(dt.datetime.today(), '%Y-%m-%d'))]
    trading_days = trading_days.apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'))

    # Obtain the portfolio changes
    df_trade = pd.DataFrame(columns = ['timestamp','equity','pnl','pnl (pct)'])

    for trade in trading_days:
        result = api.get_portfolio_history(trade)
        profit_loss = result.profit_loss
        pnl_pct = result.profit_loss_pct
        timestamp = [dt.datetime.fromtimestamp(val).strftime('%Y-%m-%d %H:%M') for val in result.timestamp]
        df_trade = pd.concat([df_trade, pd.DataFrame({'timestamp': timestamp, 'equity': result.equity, 'pnl':profit_loss, 'pnl (pct)': pnl_pct })], axis = 0)

    # Remove duplicates record with the same date
    df_trade = df_trade.sort_values(by = 'timestamp').drop_duplicates(subset = ['timestamp'],ignore_index=True, keep = 'first')

    df_trade['date'] = df_trade['timestamp'].apply(lambda x: x[:10])

    # Only query the last record of the day for intraday changes
    df_agg = df_trade.groupby('date').last()
    df_agg = df_agg.reset_index().drop('timestamp', axis = 1)

    max_drawdown = 10000 - df_agg.equity.min()
    
    return df_agg, max_drawdown

In [33]:
from alpaca_trade_api.rest import REST, TimeFrame
import pandas as pd 
import numpy as np
import os
#import pygsheets

# from ta.momentum  import RSIIndicator  
# from ta.trend import SMAIndicator
import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time 

def get_metrics(api, max_drawdown):
	acc_details = api.get_account()
	cash_balance = acc_details.cash
	portfolio_value = acc_details.portfolio_value
	acc_status = acc_details.status
	long_market_value = acc_details.long_market_value

	output = pd.DataFrame({
				'cash_balance': [cash_balance],
				'portfolio_value': [portfolio_value],
				'long_market_value': [long_market_value],
				'last_updated':  [dt.datetime.today().strftime('%Y-%m-%d %H:%M')],
				'max_drawdown': [max_drawdown]
				})

	return output

In [34]:
api.get_account()

Account({   'account_blocked': False,
    'account_number': 'PA3S324PUZ2M',
    'accrued_fees': '0',
    'balance_asof': '2022-11-23',
    'bod_dtbp': '0',
    'buying_power': '167609.18',
    'cash': '83804.59',
    'created_at': '2022-11-21T06:00:15.912605Z',
    'crypto_status': 'ACTIVE',
    'crypto_tier': 0,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'effective_buying_power': '167609.18',
    'equity': '100324.476775',
    'id': 'cb6a8e49-bfac-426c-ae92-8248b254c0a5',
    'initial_margin': '0',
    'last_equity': '100480.43',
    'last_maintenance_margin': '0',
    'long_market_value': '16519.886775',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '83804.59',
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'portfolio_value': '100324.476775',
    'position_market_value': '16519.886775',
    'regt_buying_power': '167609.18',
    'short_market_value': '0',
    'shorting_enabled

In [35]:
histpd={}
for pd in ['1D', '2D', '3D', '7D']:
    hist = api.get_portfolio_history(
        period=pd,
        extended_hours=True,
        timeframe='1D'
    ).df
    histpd[pd] = hist
histpd

{'1D':                            profit_loss  profit_loss_pct     equity
 timestamp                                                         
 2022-11-23 20:00:00-05:00    177.25575                0  100480.43,
 '2D':                            profit_loss  profit_loss_pct     equity
 timestamp                                                         
 2022-11-22 20:00:00-05:00   641.921175         0.000000  100304.71
 2022-11-23 20:00:00-05:00   177.255750         0.001767  100480.43,
 '3D':                            profit_loss  profit_loss_pct     equity
 timestamp                                                         
 2022-11-21 20:00:00-05:00  -339.090150         0.000000   99660.06
 2022-11-22 20:00:00-05:00   641.921175         0.006441  100304.71
 2022-11-23 20:00:00-05:00   177.255750         0.008220  100480.43,
 '7D':                            profit_loss  profit_loss_pct     equity
 timestamp                                                         
 2022-11-17 20:00:00-

In [36]:
portfolio_changes_output, max_drawdown = get_portfolio_changes(api)
metrics_output = get_metrics(api, max_drawdown)

AttributeError: 'str' object has no attribute 'Series'

In [ ]:
portfolio_changes_output

In [ ]:
portfolio_changes_output['pnl'].plot()

In [ ]:
portfolio_changes_output['equity'].plot()

In [ ]:
metrics_output

## Function 3 - unrealized pnl

In [ ]:
from alpaca_trade_api.rest import REST, TimeFrame
import pandas as pd 
import numpy as np
import os
#import pygsheets

# from ta.momentum  import RSIIndicator  
# from ta.trend import SMAIndicator
import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time 

def get_open_positions(api):
	open_positions = api.list_positions()

	asset_class_ls = []
	avg_entry_price_ls = []
	change_today_ls = []
	cost_basis_ls = []
	current_price_ls = []
	exchange_ls = []
	lastday_price_ls = []
	market_value_ls = []
	qty_ls = []
	side_ls = []
	symbol_ls = []
	unrealized_intraday_pl_ls = []
	unrealized_intraday_plpc_ls = []
	unrealized_pl_ls = []
	unrealized_plpc_ls = []

	for pos in open_positions:
	    asset_class_ls.append(pos.asset_class)
	    avg_entry_price_ls.append(pos.avg_entry_price)
	    change_today_ls.append(pos.change_today)
	    cost_basis_ls.append(pos.cost_basis)
	    current_price_ls.append(pos.current_price)
	    exchange_ls.append(pos.exchange)
	    lastday_price_ls.append(pos.lastday_price)
	    market_value_ls.append(pos.market_value)
	    qty_ls.append(pos.qty)
	    side_ls.append(pos.side)
	    symbol_ls.append(pos.symbol)
	    unrealized_intraday_pl_ls.append(pos.unrealized_intraday_pl)
	    unrealized_intraday_plpc_ls.append(pos.unrealized_intraday_plpc)
	    unrealized_pl_ls.append(pos.unrealized_pl)
	    unrealized_plpc_ls.append(pos.unrealized_plpc)
	    
	op_df = pd.DataFrame({'Symbol': symbol_ls, 
	                      'Asset_class': asset_class_ls, 
	                      'Qty': qty_ls,
	                      'Avg_entry_price': avg_entry_price_ls,
	                      'Change_today': change_today_ls,
	                      'Cost_basis': cost_basis_ls, 
	                      'Current_price': current_price_ls,
	                      'Exchange': exchange_ls,
	                      'Lastday_price': lastday_price_ls,
	                      'Market_value': market_value_ls,
	                      'Side': side_ls,
	                      'Unrealized_intraday_pl ($)': unrealized_intraday_pl_ls,
	                      'Unrealized_intraday_pl (%)': unrealized_intraday_plpc_ls,
	                      'Unrealized_pl ($)': unrealized_pl_ls,
	                      'Unrealized_pl (%)': unrealized_plpc_ls
	                     })

	# Convert the columns format
	op_df['Avg_entry_price'] = op_df['Avg_entry_price'].astype('float')
	op_df['Change_today'] = op_df['Change_today'].astype('float')
	op_df['Current_price'] = op_df['Current_price'].astype('float')
	op_df['Lastday_price'] = op_df['Lastday_price'].astype('float')
	op_df['Market_value'] = op_df['Market_value'].astype('float')
	op_df['Unrealized_intraday_pl ($)'] = op_df['Unrealized_intraday_pl ($)'].astype('float')
	op_df['Unrealized_intraday_pl (%)'] = op_df['Unrealized_intraday_pl (%)'].astype('float')
	op_df['Unrealized_pl ($)'] = op_df['Unrealized_pl ($)'].astype('float')
	op_df['Unrealized_pl (%)'] = op_df['Unrealized_pl (%)'].astype('float')
	op_df['Qty'] = op_df['Qty'].astype('float')

	op_df['Avg_entry_price'] = op_df.Avg_entry_price.round(2)

	op_df = op_df.sort_values(by = 'Unrealized_pl (%)', ascending = False)
	op_df['Unrealized_pl (%)'] = op_df['Unrealized_pl (%)'].mul(100)
	op_df['Unrealized_intraday_pl (%)'] = op_df['Unrealized_intraday_pl (%)'].mul(100)

	return op_df


def get_trading_cal(api):
    cal = api.get_calendar()
    cal_ls = []
    for c in cal:
        cal_ls.append(c.date)
      
    cal_ls = pd.Series(cal_ls)
    return cal_ls

def get_data(api, stock_ls, start, end = None):
    
  # first_half
    indexes = np.arange(0,len(stock_ls),5)
    for i in range(len(indexes)-1):
        data = api.get_bars(
                      symbol =  stock_ls[indexes[i]:indexes[i+1]],
                      timeframe = TimeFrame.Day,
                      start = start,
                      end = end,
                      limit = 10000,
                      adjustment= 'raw',
                      ).df
        if i == 0:
            db = data
        else:
            db = db.append(data)
    return db

# def RSI_SMA(data, symbol_list):
#     alerts = []
#     for symbol in symbol_list:
#         df = data.loc[data.symbol == symbol] # get individual stock price


#         # Calculate technical indicators
#         rsi4 = RSIIndicator(df.close, window = 4)

#         sma200 = SMAIndicator(df.close, 200)

#         df['rsi4'] = rsi4.rsi()
#         df['sma200'] = sma200.sma_indicator()
        
#         try:
#             latest = df.iloc[-1,:] # obtain latest indicators' values
#         except:
#             alerts.append([symbol,None, None])

#         alerts.append([symbol,latest.rsi4, latest.sma200])

#     alerts_df = pd.DataFrame(alerts, columns=['Symbol','RSI4','SMA200'])
#     alerts_df.set_index('Symbol', inplace = True)

#     return alerts_df

def full_df_pipeline(api):

	op_df = get_open_positions(api)

	cal_ls = get_trading_cal(api)
	tdy_date = dt.datetime.strftime(dt.datetime.today(), format = '%Y-%m-%d')
	last_trading_date = pd.Timestamp(cal_ls[cal_ls[cal_ls <= tdy_date].index[-3]]).strftime('%Y-%m-%d')
	print(last_trading_date)

	df_data = get_data(api, op_df.Symbol.unique(), start = '2019-01-01', end = last_trading_date)

	df_ta = RSI_SMA(df_data,op_df.Symbol.unique())

	full_df = op_df.join(df_ta, on = 'Symbol')
	full_df = full_df.set_index(np.arange(full_df.shape[0]))

	return full_df

In [ ]:
positions = get_open_positions(api)
positions

In [ ]:
Eqt_Screen = 99982.64



In [ ]:
Eqt_Calc = (100000+positions[ 'Unrealized_intraday_pl ($)'])
Eqt_Calc

In [ ]:
Eqt_Screen - Eqt_Calc

In [ ]:
-35.733875

In [ ]:
api.list_positions()

In [ ]:
api.list_orders()

<a id='3'></a>
# Monitoring Results - Strategy Level

* get_all_transactions_strategy : Use order id csv - done
* realized_profit_df_strategy : input order id - done
* get_open_positions_strategy : get unrealised PnL for all open positions of a strategy
* portfolio_changes_output : Get change from the initial portfolio level
* get_metrics : get all metrics using the portfolio changes output

In [ ]:
result_df = pd.DataFrame(columns = ['Qty','price','symbol','transaction_time','order_id'])

order_id_list = []

for res in result:
    order_id_list.append(res.order_id)
# pd.DataFrame(order_id_list).to_csv('Strategy_order_ids.csv', index=False) 

In [ ]:
order_ids = pd.read_csv('Strategy_order_ids.csv')['0'].values.tolist()
order_ids

In [ ]:
#api.list_orders('all', nested=True)

## Function 1 - get details of the strategy and realized PnL

In [ ]:
def get_all_transactions_strategy(api, order_ids):
    result = api.get_activities()

    result_df = pd.DataFrame()

    #print(result[0]) # cum_qty, price, symbol, transaction_time

    result_df = pd.DataFrame(columns = ['Qty','price','symbol','transaction_time','order_id'])

    order_id_list = []
    qty_list = []
    price_list = []
    symbol_list = []
    transaction_time_list = []
    type_list = []

    for res in result:
        if (res.order_id in order_ids):
            order_id_list.append(res.order_id)
            qty_list.append(res.cum_qty)
            price_list.append(res.price)
            symbol_list.append(res.symbol)
            transaction_time_list.append(res.transaction_time)
            type_list.append(res.side)



    result_df = pd.DataFrame({
        'order_id':order_id_list,
        'Qty': qty_list, 
        'Price': price_list, 
        'Symbol': symbol_list, 
        'Transaction_time': transaction_time_list, 
        'Type': type_list})

    return result_df

In [ ]:
get_all_transactions_strategy(api, order_ids )

In [ ]:
result_df = get_all_transactions_strategy(api, order_ids)

df_buy = result_df[result_df.Type == 'buy']

df_sell = result_df[result_df.Type == 'sell']

df = pd.merge(df_buy, df_sell, on = 'Symbol', how = 'right', suffixes = ['_buy','_sell'])

In [ ]:
df_buy

In [ ]:
def realized_profit_df_strategy(api, order_ids):

    result_df = get_all_transactions_strategy(api, order_ids)

    df_buy = result_df[result_df.Type == 'buy']

    df_sell = result_df[result_df.Type == 'sell']
    
    
    df = pd.merge(df_buy, df_sell, on = 'Symbol', how = 'right', suffixes = ['_buy','_sell'])
    #All the symbols which have only buy go to the unrealized list
    df_unrealized = result_df[~result_df['Symbol'].isin(df_sell.Symbol.unique())]
    #print(df_unrealized)

    testing = df_sell.sort_values(by = 'Transaction_time')

    testing['Price'] = testing.Price.astype('float')
    df_buy['Price'] = df_buy.Price.astype('float')

    output_frame = pd.DataFrame(columns = ['Symbol', 'Qty','Avg_cost','Avg_holding_period',
                                 'Earliest_buy_time','Latest_buy_time','Sell_time','Profit_per_unit','Total Profit', 'Winning_bet?'])

    for sym in testing.Symbol.unique():
        #print(sym)
        buy = df_buy.loc[df_buy.Symbol == sym]
        #All rows with sell
        sell = testing.loc[testing.Symbol == sym]
        
        obs = [] # completed sell\'s index
        for i, row in sell.iterrows():
            output_dic = {}
            if i not in obs:
                out = buy.loc[(buy.Transaction_time < row.Transaction_time)]
                print(idx)
                idx = [j for j in out.index if j not in obs]
                print(idx)
                out = out.loc[idx]
                print(out.shape[0])
                assert out.shape[0] == int(row.Qty) #Match the quantity
                
                # Avg_cost
                #print(row.Price - out.groupby('Symbol').Price.mean())
                #print(row.Transaction_time - out.groupby('Symbol').Transaction_time.mean())
                output_dict = {'Symbol': sym, 
                               'Qty': int(row.Qty), #Quantity of the sold stocks
                               'Avg_cost': round(out.groupby('Symbol').Price.mean()[0],2),
                               'Avg_holding_period': (row.Transaction_time - out.groupby('Symbol').Transaction_time.mean())[0],
                               'Earliest_buy_time': out.groupby('Symbol').Transaction_time.min()[0],
                               'Latest_buy_time': out.groupby('Symbol').Transaction_time.max()[0],
                               'Sell_time': row.Transaction_time,
                               'Profit_per_unit': round(row.Price - out.groupby('Symbol').Price.mean()[0],2),
                               'Total Profit': round((row.Price - out.groupby('Symbol').Price.mean())[0] * int(row.Qty),2),
                               'Winning_bet?': True if round(row.Price - out.groupby('Symbol').Price.mean()[0],2) > 0 else False}
                output_frame = output_frame.append(output_dict, ignore_index = True)
                
                if len(idx) > 1:
                    for ix in idx:
                        obs.append(ix)
                else:
                    obs.append(idx[0])
            

    output_frame = output_frame.sort_values('Sell_time', ascending = False)
    output_frame['Avg_holding_period'] = output_frame['Avg_holding_period'].apply(lambda x: x.days)

    return output_frame

In [ ]:
realized_profit_df_strategy(api, order_ids)

### Function 2 - portfolio changes and metrics

https://forum.alpaca.markets/t/easy-way-to-calculate-profit-per-trade-in-python/1708/6

In [ ]:
api.list_positions()

In [ ]:
unrealized_pl = (150.88 - 149.0505263157894737)*38
unrealized_pl

In [ ]:
unrealized_intraday_pl = (150.88 - 151.29)*38
unrealized_intraday_pl

In [ ]:
api.get_portfolio_history()

In [ ]:
import pandas as pd
from tabulate import tabulate
import pytz
import datetime as dt
from datetime import date

# pass the API and prevDays (0 for today, 1 since yesterday...)
def report2(api, prevDays):
    #
    # get all closed orders and import them into a dataframe
    #
    orderTotal = 500
    today = dt.date.today() - dt.timedelta(days=prevDays)
    today = dt.datetime.combine(today, dt.datetime.min.time())
    today = today.strftime("%Y-%m-%dT%H:%M:%SZ")
    #print(today)
    orders = api.list_orders(status='all', limit=orderTotal, after=today)
    if not orders:
        return
    dfOrders = pd.DataFrame()
    for o in orders:
        # convert dot notation to dict
        d = vars(o)
        # import dict into dataframe
        df = pd.DataFrame.from_dict(d, orient='index')
        # append to dataframe
        dfOrders = dfOrders.append(df, ignore_index=True)

    # select filled orders with buy or sell
    dfSel = dfOrders
    # choose a subset (use .copy() as we are slicing and to avoid warning)
    dfSel = dfSel[['submitted_at', 'filled_at', 'symbol', 'filled_qty', 'side', 'type', 'filled_avg_price', 'status']].copy()

    # convert filled_at to date
    dfSel['submitted_at'] = pd.to_datetime(dfSel['submitted_at'], format="%Y-%m-%d %H:%M:%S")
    dfSel['filled_at']    = pd.to_datetime(dfSel['filled_at'], format="%Y-%m-%d %H:%M:%S")
    # convert to our timezone
    dfSel['submitted_at'] = dfSel['submitted_at'].dt.tz_convert('America/New_York')
    dfSel['filled_at']    = dfSel['filled_at'].dt.tz_convert('America/New_York')
    # remove millis
    dfSel['submitted_at'] = dfSel['submitted_at'].dt.strftime("%Y-%m-%d %H:%M:%S")
    dfSel['filled_at']    = dfSel['filled_at'].dt.strftime("%Y-%m-%d %H:%M:%S")

    # Sort: https://kanoki.org/2020/01/28/sort-pandas-dataframe-and-series/
    # need to sort in order to perform the proper calculations
    # sort based on the following sequence of types: market then limit, then stop_limit
    dfSel['type'] = pd.Categorical(dfSel['type'], categories=["market", "limit", "stop_limit"])
    # sort first based on symbol, then type as per the list above, then submitted date
    dfSel.sort_values(by=['symbol', 'submitted_at', 'type'], inplace=True, ascending=True)

    # reset index
    dfSel.reset_index(drop=True, inplace=True)
    # drop the 'side' column
    # dfProfit = dfSel.drop('side', 1)
    dfProfit = dfSel
    # add empty 'profit' column
    dfProfit['profit'] = ''

    totalProfit = 0.0
    profitCnt   = 0
    lossCnt     = 0
    slCnt       = 0
    ptCnt       = 0
    trCnt       = 0
    qty         = 0
    profit      = 0
    sign        = {'buy': -1, 'sell': 1}

    # show header row
    #print(tabulate(dfSel[:0], headers='keys', tablefmt='simple', showindex=False))

    for index, row in dfSel.iterrows():
        # show data row
        #print(index, tabulate(dfSel[index:index+1], headers='', tablefmt='plain'))

        # conditions:
        # - buy/sell have the same symbol
        # - a trade is considered if no new/held orders are still open
        # - once qty is 0 a complete trade is confirmed and profit calculated
        # - a filled_avg_price is not None

        if index > 0:
            if dfSel['symbol'][index - 1] != dfSel['symbol'][index]:
                qty    = 0
                profit = 0

        if dfSel['status'][index] == 'held':
            continue
        if dfSel['status'][index] == 'new':
            continue
        if dfSel['filled_avg_price'][index] is None:
            continue
        if dfSel['filled_avg_price'][index] == '':
            continue
        if dfSel['filled_avg_price'][index] == 'None':
            continue

        #print(index, tabulate(dfSel[index:index+1], headers='', tablefmt='plain'))

        side      = dfSel['side'][index]
        filledQty = int(dfSel['filled_qty'][index]) * sign[side]
        qty       = qty + filledQty
        price     = float(dfSel['filled_avg_price'][index])
        pl        = filledQty * price
        profit    = profit + pl

        #print(f"{dfSel['symbol'][index]}: qty {filledQty} price {price} profit {profit:.3f}")

        if qty==0:
            # complete trade
            trCnt = trCnt + 1
            # put the profit in its column
            #dfProfit['profit'][index] = profit
            dfProfit.loc[index, 'profit'] = round(profit, 2)
            totalProfit = totalProfit + profit
            if profit >= 0:
                profitCnt = profitCnt + 1
                if dfSel['type'][index] == 'limit':
                    ptCnt = ptCnt + 1
            else:
                lossCnt = lossCnt + 1
                if dfSel['type'][index] == 'stop_limit':
                    slCnt = slCnt + 1
            profit = 0

    # append the total
    dfProfit.loc["Total", "profit"] = round(totalProfit, 2)  # dfProfit.profit.sum()

    # print profit report
    print(tabulate(dfProfit, headers='keys', tablefmt='simple', showindex=True, floatfmt=".2f"))
    #print(dfProfit.to_string())

    totalCnt = profitCnt + lossCnt
    if totalCnt > 0:
        ratio = profitCnt / totalCnt * 100.0
        print('\nProfits:', profitCnt)
        print('Losses :'  , lossCnt)
        print('Ratio  :'  , f'{ratio:.2f}%')
        print('Trades :'  , trCnt)
        print('Stops  :'  , slCnt)
        print('Targets:'  , ptCnt)

In [ ]:
report2(api, 1)

In [ ]:
def get_trading_cal(api):
    cal = api.get_calendar()
    cal_ls = []
    for c in cal:
        cal_ls.append(c.date)
      
    cal_ls = pd.Series(cal_ls)
    return cal_ls

def get_portfolio_changes_strategy(api):
    acc_start = api.get_account().created_at.strftime('%Y-%m-%d') # the date where account was created

    cal_ls = get_trading_cal(api) # obtain trading calendar

    # Obtain the trading days that were between the account created date and today, then convert to date format
    trading_days = cal_ls[(cal_ls >= acc_start) & (cal_ls < dt.datetime.strftime(dt.datetime.today(), '%Y-%m-%d'))]
    trading_days = trading_days.apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'))

    # Obtain the portfolio changes
    df_trade = pd.DataFrame(columns = ['timestamp','equity','pnl','pnl (pct)'])

    for trade in trading_days:
        result = api.get_portfolio_history(trade)
        profit_loss = result.profit_loss
        pnl_pct = result.profit_loss_pct
        timestamp = [dt.datetime.fromtimestamp(val).strftime('%Y-%m-%d %H:%M') for val in result.timestamp]
        df_trade = pd.concat([df_trade, pd.DataFrame({'timestamp': timestamp, 'equity': result.equity, 'pnl':profit_loss, 'pnl (pct)': pnl_pct })], axis = 0)

    # Remove duplicates record with the same date
    df_trade = df_trade.sort_values(by = 'timestamp').drop_duplicates(subset = ['timestamp'],ignore_index=True, keep = 'first')

    df_trade['date'] = df_trade['timestamp'].apply(lambda x: x[:10])

    # Only query the last record of the day for intraday changes
    df_agg = df_trade.groupby('date').last()
    df_agg = df_agg.reset_index().drop('timestamp', axis = 1)

    max_drawdown = 10000 - df_agg.equity.min()
    
    return df_agg, max_drawdown

In [ ]:
def get_metrics(api, max_drawdown):
	acc_details = api.get_account()
	cash_balance = acc_details.cash
	portfolio_value = acc_details.portfolio_value
	acc_status = acc_details.status
	long_market_value = acc_details.long_market_value

	output = pd.DataFrame({
				'cash_balance': [cash_balance],
				'portfolio_value': [portfolio_value],
				'long_market_value': [long_market_value],
				'last_updated':  [dt.datetime.today().strftime('%Y-%m-%d %H:%M')],
				'max_drawdown': [max_drawdown]
				})

	return output

<a id='4'></a>
# Submit Order

In [38]:
import datetime
client_id = "TradingStrategy" + "_" + str(datetime.datetime.now())

In [39]:
api.submit_order(
    symbol='AAPL',
    qty=10,
    side='buy',
    type='market',
    time_in_force='gtc',
    client_order_id=client_id)

Order({   'asset_class': 'us_equity',
    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
    'canceled_at': None,
    'client_order_id': 'TradingStrategy_2022-11-24 13:52:54.226860',
    'created_at': '2022-11-24T18:52:54.431590515Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': '9a07e29d-5562-463f-bd55-09e9b9696085',
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '10',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'source': None,
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2022-11-24T18:52:54.430489955Z',
    'subtag': None,
    'symbol': 'AAPL',
    'time_in_force': 'gtc',
    'trail_percent': None,
    'trail_price': None,
    'type': 'market',
    'updated_at': '2022-11-24T18:52:54.431590515Z

In [40]:
api = REST()
my_order = api.get_order_by_client_order_id('my_second_order')
print('Got order #{}'.format(my_order.id))

APIError: order not found for my_second_order

In [41]:
result = api.get_activities()

In [42]:
#trades = api.get_activities(date='2021-08-10', activity_types='FILL')
trades = api.get_activities()
for trade in trades:
  order_ids = trade.order_id
  price = float(trade.price)
  qty = int(trade.qty)
  side = trade.side
  symbol = trade.symbol
  client_order_id = api.get_order(trade.order_id).client_order_id
  print(client_order_id)
  alpaca_id = trade.id

TradingStrategy_2022-11-21 01:01:19.451523


<a id='5'></a>
# Running processes together


In [37]:
os.environ['APCA_API_KEY_ID'] = 'PKCXJIA0J6B064W65PG8'
os.environ['APCA_API_SECRET_KEY'] = 'e2qNpzFFqFhgOrEeSXipkCh4zi2UyVcFo2R1KKcD'
os.environ['APCA_API_BASE_URL'] = 'https://paper-api.alpaca.markets'

In [43]:
from threading import Thread

def SubmitOrder():
    print('Submitting order')    
    api = REST()
    client_id = "TradingStrategy" + "_" + str(datetime.datetime.now())
    api.submit_order(
    symbol='GBP/USD',
    qty=1,
    side='buy',
    type='market',
    time_in_force='gtc',
    client_order_id=client_id)
    time.sleep(2.4)

def func2():
    print("Getting order details")  
    api = REST()
    trades = api.get_activities()
    for trade in trades:
      order_ids = trade.order_id
      price = float(trade.price)
      qty = int(trade.qty)
      side = trade.side
      symbol = trade.symbol
      client_order_id = api.get_order(trade.order_id).client_order_id
      print(client_order_id)
      alpaca_id = trade.id
     
#     api = REST()
#     my_order = api.get_order_by_client_order_id('my_second_order')
#     print('Got order #{}'.format(my_order.id))



In [44]:
if __name__ == '__main__':    
    Thread(target = SubmitOrder).start()
    Thread(target = func2).start()

Submitting order
Getting order details


Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\tatsa\anaconda3\lib\site-packages\alpaca_trade_api\rest.py", line 234, in _one_request
    resp.raise_for_status()
  File "C:\Users\tatsa\anaconda3\lib\site-packages\requests\models.py", line 960, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 422 Client Error: Unprocessable Entity for url: https://paper-api.alpaca.markets/v2/orders

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\tatsa\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\tatsa\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\tatsa\AppData\Local\Temp\ipykernel_64736\32105472.py", line 7, in SubmitOrder
  File "C:\Users\tatsa\anaconda3\lib\site-packages\alpaca_trade_api\rest.py", line 429, in submit_order
    resp = s

TradingStrategy_2022-11-21 01:01:19.451523


In [45]:
if __name__ == '__main__':
    func1()
    func2()

NameError: name 'func1' is not defined